In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64
epochs = 1
latent_dim = 256
num_samples = 2691

data_path = 'JawiRumiCombined.txt'

ImportError: cannot import name 'experimental_functions_run_eagerly' from 'tensorflow.python.eager.def_function' (C:\Users\aiman\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\eager\def_function.py)

In [8]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
#     lines = f.read().split(':')
# for line in lines[: min(num_samples, len(lines)-1)]:
#     input_text, target_text = line.split('\t')
#     target_text = '\t' + target_text + '\n'
#     input_texts.append(input_text)
#     target_texts.append(target_text)

#     for char in input_text:
#         if char not in input_characters:
#             input_characters.add(char)
#     for char in target_text:
#         if char not in target_characters:
#             target_characters.add(char)
    lines = f.readlines()
for line in lines:
    line = line.strip()  # Remove leading/trailing whitespaces and newlines
    if line != '':
        index = line.index('.')  # Find the index of the first period
        input_text = line[:index].strip() # Extract the input text (including the period)
        target_text = line[index+1:].strip()  # Extract the output text

        target_text = '\t' + target_text + '\n'
        input_texts.append(input_text)
        target_texts.append(target_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
    

In [9]:
print(input_texts[0:2])
print(target_texts[0:2])

['', '']
['\tدي ترڠاڠا\tDia ternganga.\n', '\tتيدق مڽڠک اكن دتيڠڬلكن بڬيتو سهاج\tTidak menyangka akan ditinggalkan begitu sahaja.\n']


In [10]:
print(input_characters)
print(target_characters)
print(input_texts)
print(target_texts)

set()
{'q', 'ق', 'أ', 'D', 'G', 'J', 'k', 'ڬ', 'چ', 'l', 'r', 'ج', 'b', '1', 'L', 'K', 'ض', 'z', 'M', 'ة', 'n', 'ي', '3', 's', '5', 'ص', 'ع', 'e', 'ش', 'ء', 'ط', '7', '\t', 'ن', '’', ' ', 'Q', 'o', 'إ', 'm', 'C', 'R', 'ر', 'E', 'g', 'u', '؟', 'ح', ',', 'f', 'i', 'ف', '!', 'ه', 'j', 'ڽ', 'W', 'y', 'ز', 'و', 'س', '6', '\n', 'ڤ', 'p', 'F', 'ب', '0', 'ذ', '?', 'P', 'B', 'h', 'ݢ', 'خ', 'S', 'T', 'ى', '4', 'd', '،', '٢', 'U', 'H', 'ث', 'Z', 'غ', '8', 'w', '‘', "'", 'I', 'ت', 'Y', 'N', 't', 'ئ', 'ڠ', 'ك', 'ا', 'A', 'O', '.', 'د', '2', 'ک', 'c', 'a', '-', 'ظ', 'م', 'ی', 'ل'}
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '

In [11]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [12]:
print(len(target_texts[1890]))
print(target_texts[1890])
print(input_texts[1890])

214
	تريما كاسيه!  اڠين سڤوي٢ بهاس مڽاڤو واجه، ممندڠكن بهترا تيدق برلاير ترلالو لاجو ڤد كتيک اين	Terima kasih!  Angin sepoi-sepoi bahasa menyapu wajah, memandangkan bahtera tidak berlayar terlalu laju pada ketika ini.




In [13]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 2691
Number of unique input tokens: 0
Number of unique output tokens: 113
Max sequence length for inputs: 0
Max sequence length for outputs: 535


In [14]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)]
)
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)]
)



In [15]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')

In [16]:
print(encoder_input_data[79])
print(encoder_input_data[79].shape)
print(decoder_input_data[79])
print(decoder_input_data[79].shape)
print(decoder_target_data[79])
print(decoder_target_data[79].shape)

[]
(0, 0)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(535, 113)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(535, 113)


In [17]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

NameError: name 't' is not defined

In [ ]:
encoder_input_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 ',': 4,
 '-': 5,
 '0': 6,
 '1': 7,
 '2': 8,
 '3': 9,
 '4': 10,
 '5': 11,
 '6': 12,
 '7': 13,
 '8': 14,
 '?': 15,
 'A': 16,
 'B': 17,
 'C': 18,
 'D': 19,
 'E': 20,
 'F': 21,
 'G': 22,
 'H': 23,
 'I': 24,
 'J': 25,
 'K': 26,
 'L': 27,
 'M': 28,
 'N': 29,
 'O': 30,
 'P': 31,
 'Q': 32,
 'R': 33,
 'S': 34,
 'T': 35,
 'U': 36,
 'W': 37,
 'Y': 38,
 'Z': 39,
 'a': 40,
 'b': 41,
 'c': 42,
 'd': 43,
 'e': 44,
 'f': 45,
 'g': 46,
 'h': 47,
 'i': 48,
 'j': 49,
 'k': 50,
 'l': 51,
 'm': 52,
 'n': 53,
 'o': 54,
 'p': 55,
 'q': 56,
 'r': 57,
 's': 58,
 't': 59,
 'u': 60,
 'w': 61,
 'y': 62,
 'z': 63,
 '‘': 64,
 '’': 65}

In [ ]:
input_token_index

{' ': 0,
 '!': 1,
 "'": 2,
 '-': 3,
 '0': 4,
 '1': 5,
 '2': 6,
 '3': 7,
 '4': 8,
 '5': 9,
 '6': 10,
 '7': 11,
 '8': 12,
 '،': 13,
 '؟': 14,
 'ء': 15,
 'أ': 16,
 'إ': 17,
 'ئ': 18,
 'ا': 19,
 'ب': 20,
 'ة': 21,
 'ت': 22,
 'ث': 23,
 'ج': 24,
 'ح': 25,
 'خ': 26,
 'د': 27,
 'ذ': 28,
 'ر': 29,
 'ز': 30,
 'س': 31,
 'ش': 32,
 'ص': 33,
 'ض': 34,
 'ط': 35,
 'ظ': 36,
 'ع': 37,
 'غ': 38,
 'ف': 39,
 'ق': 40,
 'ك': 41,
 'ل': 42,
 'م': 43,
 'ن': 44,
 'ه': 45,
 'و': 46,
 'ى': 47,
 'ي': 48,
 '٢': 49,
 'چ': 50,
 'ڠ': 51,
 'ڤ': 52,
 'ک': 53,
 'ڬ': 54,
 'ڽ': 55,
 'ی': 56,
 'ݢ': 57,
 '‘': 58,
 '’': 59}

In [ ]:
encoder_input_data[0].shape

(450, 60)

In [ ]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
encoder_input_data.shape

(2691, 450, 60)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)

34/34 [==============================] - 70s 2s/step - loss: 1.3636 - accuracy: 0.7669 - val_loss: 0.8558 - val_accuracy: 0.8171


In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs )
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))

    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np. argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

            target_seq = np.zeros((1, 1, num_decoder_tokens))
            target_seq[0, 0, sampled_token_index] = 1.

            states_value = [h, c]

    return decoded_sentence

for seq_index in range(1):

    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)



1/1 [==============================] - 0s 17ms/step
-
Input sentence: دي ترڠاڠا
Decoded sentence: aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa


In [ ]:
for seq_index in range(1):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)


1/1 [==============================] - 0s 17ms/step
-
Input sentence: .دي ترڠاڠا
Decoded sentence: aaaaaaaa                                                                                                                                                                                                                                                                          
1/1 [==============================] - 0s 17ms/step
-
Input sentence: .تيدق مڽڠک اكن دتيڠڬلكن بڬيتو سهاج
Decoded sentence: aaaaaaaa                                                                                                                                                                                                                                                                          


In [ ]:
print(input_texts)
print(decoded_sentence)

['.دي ترڠاڠا', '.تيدق مڽڠک اكن دتيڠڬلكن بڬيتو سهاج', '.ڤرلاهن-لاهن دي منبركن ڤندڠن كسكليليڠ', '.اڤي راكوس يڠ ممباكر همڤير كسموا رومه دڤركامڤوڠن ڬاڤورا مولاءي جينق، ڤرلاهن-لاهن ترڤادم', '.سبهاڬين سهاج ماسيه تڬر مليڠ-ليوق، منجيلت ڤنوه سراكه ڤاڤن دان كايو رومه', '.رات٢ رومه دڤركامڤوڠن رنتوڠ دان روبوه سمواڽ', '.حيوان ترنقن باڽق يڠ ماتي دان سلبيهڽ برتمڤيارن لاري', '.دمكين جوݢ كودا٢ ميليق ڤندودوق كامڤوڠ', '.تيڠڬل دي دان جسد٢ تنڤا ڽاوا يڠ برڬليمڤڠن', '.ڤندڠنڽ ترهنتي ڤد جسد ايهندا دان بوندا', '.اءير ماتڽ برجوراين كمبالي', '.دي چوبا مناهن سدو، ممبواتكن توبوهڽ برڬڬر دهشت', '.جيواڽ باڬاي دتيكام-تيكم', '.لوتوتڽ كمودين منجادي لمه، دان كاكيڽ ملوڠلاي', '.تروس دي ترجليڤوق دان ملوتوت کتانه', '.داهي برتمو لنتاي بومي', '.دي مناڠيس سماهو-ماهوڽ', '.لاما دي بركأداءن سدميكين', '.دي مناڠيس سهيڠݢ اءير مات تيدق لاڬي برسيسا', '.سهيڠݢ تياد لاڬي اڤي يڠ منجيلتي رومه', '.سهيڠݢ ظلمة ممبالوتي لاڠيت مالم، دان كڬلاڤن مڽليموتي بوانا', '.برتمنكن سيسا بارا٢ اڤي، دي تيدق برڬرق-ڬرق', '.هاڽ ممباتو دالم كأداءن برسجود', '.صلاة 